# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.4/11.6 MB 4.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.6 MB 6.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/11.6 MB 7.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.6 MB 7.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.6 MB 8.6 MB/s eta 0:00:02
   ---------- ----------------------------- 3.0/11.6 MB 9.2 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.6 MB 9.6 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 9.9 MB/s eta 0:00:01
   ---------------- ----------------------- 4.8/11.6 MB 10.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.3/11.6 MB 10.3 MB/s eta 0:00:01
   ----------

In [3]:
#Imports
import numpy as np
import torch
import pandas as pd

In [4]:
#visualising training data
# TODO: put data in drive?
train_data = pd.read_json("data/train-claims.json")
train_data = train_data.transpose()
train_data.head()


,claim_text,claim_label,evidences
claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1..."
claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]"
claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]"
claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5..."
claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72..."


In [5]:
#visualising evidence data
evidence = pd.read_json("data/evidence.json",typ='series')

In [6]:
print(len(evidence))
evidence.head()

1208827


evidence-0    John Bennet Lawes, English entrepreneur and ag...
evidence-1    Lindberg began his professional career at the ...
evidence-2    ``Boston (Ladies of Cambridge)'' by Vampire We...
evidence-3    Gerald Francis Goyer (born October 20, 1936) w...
evidence-4    He detected abnormalities of oxytocinergic fun...
dtype: object

In [7]:

from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
evidence_pd = pd.DataFrame(evidence).reset_index()
evidence_pd.columns = ['claim_id', 'claims']


In [9]:

evidence_pd.head()

,claim_id,claims
0,evidence-0,"John Bennet Lawes, English entrepreneur and ag..."
1,evidence-1,Lindberg began his professional career at the ...
2,evidence-2,``Boston (Ladies of Cambridge)'' by Vampire We...
3,evidence-3,"Gerald Francis Goyer (born October 20, 1936) w..."
4,evidence-4,He detected abnormalities of oxytocinergic fun...


In [10]:
import re
evidence_pd["pro_evidence"] = evidence_pd["claims"].apply(str.lower)
evidence_pd["pro_evidence"] = evidence_pd["pro_evidence"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
evidence_pd["pro_evidence"] = evidence_pd["pro_evidence"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))


In [11]:
evidence_pd

,claim_id,claims,pro_evidence
0,evidence-0,"John Bennet Lawes, English entrepreneur and ag...",john bennet lawes english entrepreneur agricul...
1,evidence-1,Lindberg began his professional career at the ...,lindberg began professional career age eventua...
2,evidence-2,``Boston (Ladies of Cambridge)'' by Vampire We...,boston lady cambridge vampire weekend
3,evidence-3,"Gerald Francis Goyer (born October 20, 1936) w...",gerald francis goyer born october professional...
4,evidence-4,He detected abnormalities of oxytocinergic fun...,detected abnormality oxytocinergic function sc...
...,...,...,...
1208822,evidence-1208822,Also on the property is a contributing garage ...,also property contributing garage apartment
1208823,evidence-1208823,| class = ``fn org'' | Fyrde | | | | 6110 | | ...,class fn org fyrde volda
1208824,evidence-1208824,"Dragon Storm (game), a role-playing game and c...",dragon storm game roleplaying game collectible...
1208825,evidence-1208825,It states that the Zeriuani ``which is so grea...,state zeriuani great realm tradition relates t...


In [12]:
i = "evidence-1208822"
evidence_pd.iloc[int(re.sub("[^0-9]", "", i))]["pro_evidence"]

'also property contributing garage apartment'

In [13]:

train_data["pro_claim_text"] = train_data['claim_text'].apply(str.lower)
train_data["pro_claim_text"] = train_data["pro_claim_text"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
train_data["pro_claim_text"] = train_data["pro_claim_text"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))

train_data["pro_evidences"] = train_data["evidences"].apply(lambda x:  [evidence_pd.iloc[int(re.sub("[^0-9]", "", i))]["pro_evidence"] for i in x])
train_data["evidence_ids"] = train_data["evidences"].apply(lambda x:  [int(re.sub("[^0-9]", "", i)) for i in x])
train_data.head()

,claim_text,claim_label,evidences,pro_claim_text,pro_evidences,evidence_ids
claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1...",scientific evidence pollutant higher concentra...,[high concentration time atmospheric concentra...,"[442946, 1194317, 12171]"
claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]",el niño drove record high global temperature s...,[climate change due natural force human activi...,"[338219, 1127398]"
claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]",pdo switched cool phase,[evidence reversal prevailing polarity meaning...,"[530063, 984887]"
claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5...",weather channel cofounder john coleman provide...,[convincing scientific evidence human release ...,"[1177431, 782448, 540069, 352655, 1007867]"
claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72...",january capped month period global temperature...,"[average temperature c f, iranian persian cale...","[1010750, 91661, 722725, 554161, 430839]"


In [14]:
train_data["pro_evidences"][0][1]

C:\Users\Asura\AppData\Local\Temp\ipykernel_5928\3889600003.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_data["pro_evidences"][0][1]


'plant grow much percent faster concentration ppm co compared ambient condition though assumes change climate limitation nutrient'

In [16]:
dev = pd.read_json("data/dev-claims.json")
dev = dev.transpose()
dev.head()
dev["pro_claim_text"] = dev['claim_text'].apply(str.lower)
dev["pro_claim_text"] = dev["pro_claim_text"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
dev["pro_claim_text"] = dev["pro_claim_text"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))

dev["pro_evidences"] = dev["evidences"].apply(lambda x:  [evidence_pd.iloc[int(re.sub("[^0-9]", "", i))]["pro_evidence"] for i in x])
dev.head()

,claim_text,claim_label,evidences,pro_claim_text,pro_evidences
claim-752,[South Australia] has the most expensive elect...,SUPPORTS,"[evidence-67732, evidence-572512]",south australia expensive electricity world,[citation needed south australia highest retai...
claim-375,when 3 per cent of total annual global emissio...,NOT_ENOUGH_INFO,"[evidence-996421, evidence-1080858, evidence-2...",per cent total annual global emission carbon d...,[unep green economy report state aagricultural...
claim-1266,This means that the world is now 1C warmer tha...,SUPPORTS,"[evidence-889933, evidence-694262]",mean world warmer preindustrial time,[multiple independently produced instrumental ...
claim-871,"“As it happens, Zika may also be a good model ...",NOT_ENOUGH_INFO,"[evidence-422399, evidence-702226, evidence-28...",happens zika may also good model second worryi...,[genetic disorder result deleterious mutation ...
claim-2164,Greenland has only lost a tiny fraction of its...,REFUTES,"[evidence-52981, evidence-264761, evidence-947...",greenland lost tiny fraction ice mass,[iceberg calving happened average greenland lo...


In [18]:
test = pd.read_json("data/test-claims-unlabelled.json")
test = test.transpose()

test["pro_claim_text"] = test['claim_text'].apply(str.lower)
test["pro_claim_text"] = test["pro_claim_text"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test["pro_claim_text"] = test["pro_claim_text"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))
test.head()


,claim_text,pro_claim_text
claim-2967,The contribution of waste heat to the global c...,contribution waste heat global climate
claim-979,“Warm weather worsened the most recent five-ye...,warm weather worsened recent fiveyear drought ...
claim-1609,Greenland has only lost a tiny fraction of its...,greenland lost tiny fraction ice mass
claim-1020,“The global reef crisis does not necessarily m...,global reef crisis necessarily mean extinction...
claim-2599,Small amounts of very active substances can ca...,small amount active substance cause large effect


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
import math
import copy
from collections import Counter
import tqdm


In [20]:
#words = train['pro_claim_text'] + evidence_pd["pro_evidence"]
train_words = (train_data['pro_claim_text'].apply(lambda x : x.split())).tolist()
evidence_words = (evidence_pd["pro_evidence"].apply(lambda x : x.split())).tolist()

In [21]:
evidence_words[0]

['john',
 'bennet',
 'lawes',
 'english',
 'entrepreneur',
 'agricultural',
 'scientist']

14558

In [22]:
train_words1 = [x for xs in train_words for x in xs ]
evidence_words1 = [x for xs in evidence_words for x in xs ]
vocab = list(set(train_words1 + evidence_words1))
word_counts = Counter(train_words1 + evidence_words1)

In [ ]:

len(word_counts)

637640

In [23]:

id2label = {0:"SUPPORTS", 1:"NOT_ENOUGH_INFO", 2:"REFUTES", 3:"DISPUTED"}
label2id = {"SUPPORTS":0, "NOT_ENOUGH_INFO":1, "REFUTES":2, "DISPUTED":3}
id2token = {0:"<pad>", 1:"<cls>", 2:"<sep>", 3:"<unk>"}
token2id = {"<pad>": 0, "<cls>": 1, "<sep>": 2, "<unk>": 3}


In [24]:
word_counts["plant"]

8298

In [25]:
idx = 4
for i, j in word_counts.items():
  if j > 5:
    id2token[idx] = i
    token2id[i] = idx
    idx +=1



In [26]:
def wordtointvec(inputs, token2id):
  lst = []
  for i in inputs:

    sent = []
    for w in i.split(" "):
      sent.append(token2id.get(w, token2id["<unk>"]))
    lst.append(sent)

  return lst
def wordtointvec4evi(inputs, token2id):
  lst = []
  for x in inputs:
    sent = []
    for i in x:
      insent = []
      for w in i.split(" "):
        insent.append(token2id.get(w, token2id["<unk>"]))
      sent.append(insent)

    lst.append(sent)

  return lst


In [27]:
train_data['pro_claim_text']

claim-1937    scientific evidence pollutant higher concentra...
claim-126     el niño drove record high global temperature s...
claim-2510                              pdo switched cool phase
claim-2021    weather channel cofounder john coleman provide...
claim-2449    january capped month period global temperature...
                                    ...                        
claim-1504    climate scientist say aspect case hurricane ha...
claim-243     assessment report ipcc estimated human emissio...
claim-2302    since mid global temperature warming around de...
claim-502     abnormal temperature spike february earlier mo...
claim-3093    sending oscillating microwave antenna inside v...
Name: pro_claim_text, Length: 1228, dtype: object

In [28]:
train_vec = wordtointvec(train_data['pro_claim_text'],token2id)
train_evi_vec = wordtointvec4evi(train_data['pro_evidences'],token2id)
#evi_vec = wordtointvec(evidence_pd["pro_evidence"],token2id)
dev_vec = wordtointvec(dev['pro_claim_text'],token2id)


In [29]:

print(train_evi_vec[0])

[[20, 8, 52, 110, 8, 851, 278, 279, 1843, 14, 15, 2421, 8, 765, 7, 1001, 3226, 2957, 19214, 3, 8271, 38097, 154], [13, 237, 95, 299, 1791, 8, 765, 4353, 103, 22355, 170, 784, 10925, 122, 121, 7018, 20717], [7, 278, 279, 8, 22501, 894, 13, 2116, 6789, 371]]


In [30]:
train_data['pro_evidences'][0]

C:\Users\Asura\AppData\Local\Temp\ipykernel_5928\4177359292.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_data['pro_evidences'][0]


['high concentration time atmospheric concentration greater carbon dioxide toxic animal life raising concentration ppm higher several hour eliminate pest whitefly spider mite greenhouse',
 'plant grow much percent faster concentration ppm co compared ambient condition though assumes change climate limitation nutrient',
 'higher carbon dioxide concentration favourably affect plant growth demand water']

In [31]:
sum([len(i) for i in train_vec])/ len(train_vec)

11.57328990228013

In [32]:
sum([len(i) for i in train_vec])/ len(train_vec)

11.57328990228013

In [33]:
def addpadding(l, inputs, token2id):
  lst = []
  for i in inputs:
    if len(i) < l:
      lst.append([token2id["<cls>"]]+ i + [token2id["<sep>"]] + (l-len(i)) * [token2id["<pad>"]])
    else:

      lst.append([token2id["<cls>"]]+ i[:l] + [token2id["<sep>"]] )
  return lst


In [34]:
train_in = addpadding(34, train_vec, token2id)
#train_evi_in = addpadding(100, train_evi_vec, token2id)
#evi_in = addpadding(100, evi_vec, token2id)
dev_in =addpadding(34, dev_vec, token2id)


In [35]:
max_len = 280
for i in range(len(train_in)):
  for j in train_evi_vec[i]:
    train_in[i].append(token2id["<sep>"])
    train_in[i].extend(j)
  train_in[i].append(token2id["<sep>"])
  if len(train_in[i]) < max_len:

    train_in[i].extend([token2id["<pad>"]] * (max_len - len(train_in[i])))

print(train_in[0])

[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 20, 8, 52, 110, 8, 851, 278, 279, 1843, 14, 15, 2421, 8, 765, 7, 1001, 3226, 2957, 19214, 3, 8271, 38097, 154, 2, 13, 237, 95, 299, 1791, 8, 765, 4353, 103, 22355, 170, 784, 10925, 122, 121, 7018, 20717, 2, 7, 278, 279, 8, 22501, 894, 13, 2116, 6789, 371, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [36]:
dev_label = [label2id[i] for i in dev["claim_label"]]

In [37]:
type(dev_label)

list

In [38]:
for i in train_in:
  if len(i)>280:
    print(i)

In [39]:
from torch.utils.data import DataLoader
class TrainDataset(Dataset):
  def __init__(self, text_data, label_data):
    self.text_data = text_data
    self.label_data = [label2id[i] for i in label_data]
    

  def __getitem__ (self, index):
    return [self.text_data[index], self.label_data[index]]
  def __len__(self):
    return len(self.text_data)

  def collate_fn(self, data):
    q = []
    evi = []
    labels = []
    for sen, lab in data:
      q.append(sen)
      labels.append(lab)
    batch_encoding = {}
    batch_encoding["queries"] =  torch.LongTensor(q)

    batch_encoding["labels"] = torch.LongTensor(labels)
    return batch_encoding


In [41]:
train_dataset = TrainDataset(train_in, train_data["claim_label"])
dataloader = DataLoader(train_dataset, batch_size = 20, shuffle = True, num_workers = 0,collate_fn=train_dataset.collate_fn)


In [42]:
len(dataloader)

62

In [43]:
for (i, targets) in enumerate(dataloader):
  print(targets["queries"])

tensor([[   1, 1802,  882,  ...,    0,    0,    0],
        [   1,  371,  111,  ...,    0,    0,    0],
        [   1,  121, 1452,  ...,    0,    0,    0],
        ...,
        [   1,  840, 1318,  ...,    0,    0,    0],
        [   1,  394,  789,  ...,    0,    0,    0],
        [   1,  469, 1709,  ...,    0,    0,    0]])
tensor([[  1, 160, 402,  ...,   0,   0,   0],
        [  1, 318, 241,  ...,   0,   0,   0],
        [  1, 406, 774,  ...,   0,   0,   0],
        ...,
        [  1, 954, 209,  ...,   0,   0,   0],
        [  1, 209, 470,  ...,   0,   0,   0],
        [  1, 192, 923,  ...,   0,   0,   0]])
tensor([[   1, 1075, 1522,  ...,    0,    0,    0],
        [   1,  369, 2367,  ...,    0,    0,    0],
        [   1,  502,  503,  ...,    0,    0,    0],
        ...,
        [   1, 2310,  543,  ...,    0,    0,    0],
        [   1,  275,  312,  ...,    0,    0,    0],
        [   1, 1018,  977,  ...,    0,    0,    0]])
tensor([[   1,  371,  372,  ...,    0,    0,    0],
      

In [44]:
class AttentionBlock(nn.Module):
  def __init__(self, embed_dim, n_heads, drop_prob):
    super(AttentionBlock , self).__init__()
    self.att = nn.MultiheadAttention(embed_dim, n_heads)
    self.ffn = nn.Sequential(
        nn.Linear(embed_dim, 4*embed_dim),
        nn.ReLU(),
        nn.Linear(4*embed_dim, embed_dim)
    )
    self.ln1 = nn.LayerNorm(embed_dim, eps = 1e-6)
    self.ln2 = nn.LayerNorm(embed_dim, eps = 1e-6)
    self.dropout1 = nn.Dropout(drop_prob)
    self.dropout2 = nn.Dropout(drop_prob)
  def forward(self, inputs):
    attn_out, _ = self.att(inputs, inputs,inputs)
    attn_out = self.dropout1(attn_out)
    out1 = self.ln1(inputs + attn_out)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    return self.ln2(out1 + ffn_output)
class ClassificationModel(nn.Module):
  def __init__(self, embed_dim, wordlist_size, block_size, drop_prob, n_classes, n_heads):
    super(ClassificationModel , self).__init__()
    self.token_embed = nn.Embedding(wordlist_size, embed_dim)
    self.pos_embed = nn.Embedding(block_size, embed_dim)
    self.AttentionBlock = AttentionBlock(embed_dim, n_heads, drop_prob)
    self.ln1 = nn.LayerNorm(embed_dim)
    self.ln2 = nn.LayerNorm(embed_dim)
    self.classifier = nn.Sequential(
        nn.Linear(embed_dim, embed_dim),
        nn.ReLU(),
        nn.Dropout(drop_prob),
        nn.ReLU(),
        nn.Linear(embed_dim, n_classes),
        nn.Softmax(dim = 1)
    )
  def forward(self, token):
    r,c = token.shape
    emb = self.token_embed(token)
    emb = emb+ self.pos_embed(torch.arange(c, device = device))
    out = self.AttentionBlock(emb)
    out = out.mean(dim = 1)
    out = self.classifier(out)
    return out

In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classification_model = ClassificationModel(embed_dim = 512, wordlist_size = len(id2token), block_size = 280, drop_prob = 0.1, n_classes = 4, n_heads = 4)
classification_model.to(device)
lr = 0.0001
optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr)
loss_t = nn.CrossEntropyLoss()


In [77]:
#import time
def train():
  classification_model.train()
  total_loss = 0.
  #start_time = time.time()
  for (i, targets) in enumerate(dataloader):
    optimizer.zero_grad()
    out = classification_model(targets["queries"].cuda())
    loss = loss_t(out, targets["labels"].cuda())
    #loss = loss
    #torch.nn.utils.clip_grad_norm_()

    loss.backward()
    optimizer.step()
    total_loss += loss.item()


def evaluate(model, input, out):
    lst = []
    start = 0
    batch_size = 50
    in_len = len(input[0])
    model.eval()
    correct_count = 0
    while start < len(out):
        end = min(start+ batch_size, len(out))

        model_input = torch.LongTensor(input[start:end]).view(-1, in_len).cuda()

        model_output = model(model_input)
        model_output = torch.argmax(model_output, 1).tolist()

        for i, j in zip(model_output, out[start: end]):
            if i == j:
                correct_count += 1

        start = end
    lst = correct_count / len(out)
    print("\n")
    print("Classification Accuracy: %.3f" % lst)
    print("\n")

    model.train()
    return lst





In [78]:

epochs = 10
for epoch in range(epochs):
  loss = 0
  train()
  f_score = evaluate(classification_model, dev_in, dev_label)
  print("fscore", f_score)



Classification Accuracy: 0.357


fscore 0.35714285714285715


Classification Accuracy: 0.370


fscore 0.37012987012987014


Classification Accuracy: 0.344


fscore 0.34415584415584416


Classification Accuracy: 0.409


fscore 0.4090909090909091


Classification Accuracy: 0.377


fscore 0.37662337662337664


Classification Accuracy: 0.422


fscore 0.42207792207792205


Classification Accuracy: 0.409


fscore 0.4090909090909091


Classification Accuracy: 0.416


fscore 0.4155844155844156


Classification Accuracy: 0.409


fscore 0.4090909090909091


Classification Accuracy: 0.422


fscore 0.42207792207792205


In [61]:
def predict( dev_input, model):
    lst = []
    start = 0
    batch_size = 50
    in_len = len(dev_input[0])
    model.eval()

    
    
    while start < len(dev_input):
        end = min(start+ batch_size, len(dev_in))
        
        model_input= torch.LongTensor(dev_input[start:end]).view(-1, in_len).cuda()
        
        model_output = model(model_input)
        model_output = torch.argmax(model_output, 1).tolist()
        lst.extend(model_output)
        
        start = end

    return lst


In [71]:
dev_classes = predict(dev_in, classification_model)

In [72]:
dev_classes

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0]

In [ ]:
# class MultiHeadAttention(nn.Module):
#   def __init__(self , d_model, nhead):
#     #d_model / nhead should equal to 0
#     super(MultiHeadAttention, self).__init__()
#     self.d_model = d_model
#     self.d_model_type = 'Transformer'
#     self.nhead = nhead
#     self.d_k = d_model//nhead
#     self.Q = nn.Linear(d_model, d_model)
#     self.K = nn.Linear(d_model, d_model)
#     self.V = nn.Linear(d_model, d_model)
#     self.O = nn.Linear(d_model, d_model)
#   def scaled_dot_prod_attention(self, Q,K,V,mask = None):
#     attention_score = torch.matmul(Q,K.transpose(-2,-1))/ math.sqrt(self.d_k)
#     if mask:
#       attention_score = attention_score.masked_fill(mask == 0, -1e9)
#       attention_probs = torch.softmax(attention_score, dim = -1)
#       out = torch.matmul(attention_probs, V)
#       return out
#   def split_heads(self, n):
#     bat_size, seq_len, d_model = n.size()
#     return n.view(bat_size, seq_len, self.nhead, self.K).transpose(1,2)

#   def combine_heads(self, n):
#     bat_size, _, seq_len, d_k = n.size()
#     return n.transpose(1,2).contiguous().view(bat_size, seq_len, self.d_model)


#   def forward(self, Q,K,V,mask = None):
#     Q = self.split_heads(self.Q(Q))
#     K = self.split_heads(self.K(K))
#     V = self.split_heads(self.V(V))
#     attention_out = self.scaled_dot_prod_attention(Q,K,V,mask)
#     output = self.O(self.combine_heads(attention_out))
#     return output


In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, dropout = 0.1, max_seq_len = 5000):
    super(PositionalEncoding, self).__init__()
    #self.dropout = nn.Dropout(p=dropout)

    pe = torch.zero(max_seq_len, d_model)
    pos = torch.arange(0, max_seq_len, dtype = torch.float).unsqueeze(1)
    div = torch.exp(torch.arange(0,d_model, 2).float() * -(math.log(10000.0)/d_model))
    pe[:, 0::2] = torch.sin(pos * div)
    pe[:, 1::2] = torch.cos(pos * div)
    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    # x = x + self.pe[:x.size(0), :]
    # return self.dropout(x)
    return x + self.pe[:, :x.size(1)]

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        output = self.fc(enc_output)
        return output

# Two steps for the this task
# first. find all relavent evidence, either use contextual embedding or similarity scoring
# second. classify the evidents into 4 classes.

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*